# HPC Test Report

This is a *proof of concept* for a HPC test results report. It is part of the [JDP](https://rpalethorpe.io.suse.de/jdp/) project. This report is created using Jupyter and Julia, for details see the (highly annotated) general [kernel group report](https://rpalethorpe.io.suse.de/jdp/reports/Report-DataFrames.html).

## Index

1. [Setup](#Setup)
2. [Results](#Results)

## Setup

First we need to build up our data structures to create the test matrix. There are some stats here which may be useful, but otherwise you can safely skip this part most of the time.

In [ ]:
# Monitors library source files and recompiles them after most changes
import Revise

# Run the init script which will setup the JDP project if necessary
include("../src/init.jl")

# Bring DataFrame's _members_ into our namespace, so we can call them directly
using DataFrames

# import the markdown string literal/macro
import Markdown: @md_str

# Import some libraries from the JDP project
using JDP.Conf
using JDP.Trackers.OpenQA    # Contains functions for dealing with the OpenQA web API
using JDP.Trackers.Bugzilla  # Functions for accessing the Bugzilla API(s)
using JDP.Repository

In [ ]:
allres = Repository.fetch(OpenQA.TestResult, Vector, "osd"; refresh=true, groupid=130)

md"We have **$(length(allres))** results in total"

In [ ]:
allhpcres = filter(allres) do res
    get(res.suit, 2, nothing) == "HPC"
end

md"We have **$(length(allhpcres))** HPC test module results"

In [ ]:
product = "sle-15-SP1-Installer-DVD"

hpcres = filter(allhpcres) do res
    res.product == product
end

md"We have **$(length(hpcres))** HPC test results for $product"

In [ ]:
builds = map(res -> res.build, hpcres) |> unique
builds = map(b -> (parse(Float64, b), b), builds)
sort!(builds, by=(b -> b[1]))

totalbuilds = length(builds)
recentnum = max(0, min(6, totalbuilds - 6))
recentbuilds = join(map(b -> "**$(b[2])**", builds[end-recentnum+1:end]), ", ", " and ")

md"We have **$totalbuilds** builds in total. The last $recentnum are $recentbuilds"

In [ ]:
headers = vcat([Symbol("Job name"), Symbol("Module name")], 
    map(b -> Symbol(b[2]), builds[end-recentnum:end]))
testnames = map(res -> (res.suit[3], res.name), hpcres) |> unique |> sort

buildsres = Dict{String, Dict{Tuple{String, String}, Union{Nothing, OpenQA.TestResult}}}(
    build[2] => Dict(name => nothing for name in testnames) for 
        build in builds[end-recentnum:end]
)

for res in Iterators.filter(res -> haskey(buildsres, res.build), hpcres)
    name = (res.suit[3], res.name)
    buildres = buildsres[res.build]
    if buildres[name] == nothing || buildres[name].result != "passed"
        buildres[name] = res
    end 
end

failed_testnames = []
for name in testnames
    boring = true
    
    for build in keys(buildsres)
        res = buildsres[build][name] 
        if res == nothing || res.result != "passed"
            boring = false
        end
    end
    
    if boring
        for build in keys(buildsres)
            delete!(buildsres[build], name)
        end
    else
        push!(failed_testnames, name)
    end
end

buildcols = [[let res = buildsres[build[2]][name]
    res == nothing ? "none" : res.result
end for name in failed_testnames] for build in builds[end-recentnum:end]]

md"""
Ignoring **$(length(testnames) - length(failed_testnames))** of **$(length(testnames))** tests
because they only had pass results.
"""

## Results

Below is a matrix of the HPC results. Test scenarious which always pass have been removed.

In [ ]:
withenv("LINES" => 100) do
display(DataFrame([map(t -> t[1], failed_testnames), map(t -> t[2], failed_testnames), buildcols...], 
            headers))
end